# Tutorial: Cross-Language Word Embeddings

We have been previosuly representing individual words as features. This assignment focuses on representation of words in continuous domain w.r.t context. Word2Vec has 2 different methods for learning representation: 


1.   Bag of words: The objective is to predict middle word based on the surrounding words. Thus a word is represented in terms of the context of its occurence.
2.   Skip-gram model: The objective is to predict the surrounding words (context) based on the input word.

Genism provides efficient implmenetation of models to generate vector embeddings.

We're also going to look at embeddings generated for different languages with Facebooks's library FastText.


In [2]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [4]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]
# Sample sentence from Shakespearean play
' '.join(lines[0])

--2021-04-08 20:50:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.2’

shakespeare_plays.t 100%[===================>]   4.53M  3.11MB/s    in 1.5s    

2021-04-08 20:50:10 (3.11 MB/s) - ‘shakespeare_plays.txt.2’ saved [4746840/4746840]



'alls well that ends well by william shakespeare dramatis personae king of france the duke of florence bertram count of rousillon lafeu an old lord parolles a'

Then, we'll estimate a simple word2vec model on the Shakespeare texts. You can try passing the vector size and skip window as well here.

In [5]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [6]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8456141948699951),
 ('prince', 0.7579331994056702),
 ('york', 0.7577690482139587),
 ('duke', 0.7525829076766968),
 ('clarence', 0.7353566288948059),
 ('son', 0.7147427797317505),
 ('warwick', 0.7145901918411255),
 ('gloucester', 0.7142559289932251),
 ('princess', 0.7119390368461609),
 ('suffolk', 0.7051541805267334)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [7]:
model.wv.most_similar(positive=['othello'])

[('desdemona', 0.961658775806427),
 ('cleopatra', 0.9296777844429016),
 ('troilus', 0.9229164719581604),
 ('iago', 0.9221736192703247),
 ('ham', 0.9135774970054626),
 ('glou', 0.9131096005439758),
 ('emilia', 0.9123107194900513),
 ('lear', 0.9111637473106384),
 ('valentine', 0.9066769480705261),
 ('pisanio', 0.9043502807617188)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [8]:
model.wv.similarity('othello', 'desdemona')

0.9616587

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object. Cosine similarity will tell you cosine of the angle between two vectors. With a high cosine distance between 2 vectors [ranging between 0 and 1], it can be said that there's a similarity. Since, the dimension of the vectors is 100 for the Word2Vec model and 300 for FastText, you'll use numpy to calculate cosine distance between the two Word2Vec vectors or 2 FastText vectors.

In [ ]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return 0

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [9]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2021-04-08 21:05:37--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  15.5MB/s    in 5.4s    

2021-04-08 21:05:43 (12.0 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2021-04-08 21:05:43--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M  15.6MB/s    in 5.4s    

2021-04-08 21:05:49 (12.0 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]



We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [16]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')
# Sample English word 'on' and its vector embedding
print(list(envec.items())[15])
# Sample French word 'du' and its vector embedding
print(list(frvec.items())[15])

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

('on', array([-0.0108,  0.0298, -0.0147,  0.0224, -0.0068, -0.0253,  0.0375,
       -0.0396, -0.019 , -0.0771, -0.0037,  0.004 ,  0.0159, -0.0456,
        0.0498, -0.0383, -0.0111, -0.0132,  0.0137,  0.0515, -0.0573,
       -0.0037, -0.11  ,  0.0006, -0.0783, -0.0704,  0.0562, -0.0938,
        0.0244,  0.002 ,  0.0066,  0.0976, -0.0229,  0.0182, -0.0141,
       -0.0808, -0.0776,  0.0101,  0.0992, -0.069 ,  0.0298,  0.0239,
       -0.0178,  0.0447, -0.0796, -0.0088,  0.0033,  0.0143,  0.0109,
       -0.1093,  0.0214, -0.067 ,  0.0189,  0.0015,  0.0077, -0.0294,
       -0.0046,  0.0186, -0.0374,  0.0908, -0.003 , -0.0764,  0.0695,
       -0.0898, -0.0616, -0.0906, -0.0102,  0.0035,  0.0037,  0.0754,
        0.0338,  0.0165, -0.0389, -0.0462,  0.0064,  0.0059,  0.1263,
        0.085 ,  0.0313,  0.0005,  0.0274,  0.0103,  0.0673,  0.035 ,
        0.0257, -0.072 , -0.0545, -0.0663,  0.0887, -0.0877, -0.0359,
        0.0791,  0.102 , -0.0959,  0.0649,  0.0506,  0.0088, -0.0386,
       -0.045

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible. For computational efficiency, you can calculate cosine for all vectors w.r.t a given vector by casting the first vector as a 1D numpy array and the 2nd vector as a 2D numpy array. You can then call `np.max` and `np.argmax` to get the most similar word and corresponding similarity. You can do this by adding some parameters to your cosim function to check if the computation is supposed to be for 2 vectors or 2 arrays.

In [ ]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [ ]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-04-14 03:22:37--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  3.66MB/s    in 0.4s    

2020-04-14 03:22:37 (3.66 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [ ]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

First select tuples from links for the language you're evaluating. For French, `words_lang = all words that satisfy condition: links[i][1] == 'fr'`.

**Baseline Accuracy** = `sum(words_lang[i][Index for English term] == words_lang[i][Index for term in that language])/len(words_lang)`

For word i in words, get the most similar word based on cosine similarity for FastText embeddings.
`similar_word, similarity = mostsimlar(envec(words_lang[i][Index for English term], frvec)`

**Accuracy** = `sum(similar_word == words_lang[i][Index for term in that language]))/len(words_lang)`

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy = 0
baselineAccuracy = 0

**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".